In [1]:
import pandas as pd

df = pd.read_csv('/kaggle/input/patent-phrase/train.csv')

df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [2]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [3]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [4]:
!pip install transformers

In [5]:
!pip install -U datasets

In [6]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [7]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [8]:
model_nm = 'microsoft/deberta-v3-small'

In [9]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
tokz.tokenize("G'day folks, I'm Jeremy from fast.ai!")

['▁G',
 "'",
 'day',
 '▁folks',
 ',',
 '▁I',
 "'",
 'm',
 '▁Jeremy',
 '▁from',
 '▁fast',
 '.',
 'ai',
 '!']

In [11]:
def tok_func(x): return tokz(x["input"])

In [12]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [13]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [14]:
tokz.vocab['▁of']

265

In [15]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [16]:
eval_df = pd.read_csv('/kaggle/input/patent-phrase/test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


In [17]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [18]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [19]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [20]:
from transformers import TrainingArguments,Trainer
import numpy as np

In [21]:
bs = 128
epochs = 4
lr = 8e-5

In [22]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-23-48e257fff1ad>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.035693,0.797464
2,No log,0.023886,0.820960
3,0.030700,0.021750,0.830964
4,0.030700,0.022456,0.833004


TrainOutput(global_step=856, training_loss=0.02352284605257979, metrics={'train_runtime': 261.5392, 'train_samples_per_second': 418.354, 'train_steps_per_second': 3.273, 'total_flos': 715555561923540.0, 'train_loss': 0.02352284605257979, 'epoch': 4.0})

In [25]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.5703125 ],
       [ 0.72070312],
       [ 0.55126953],
       [ 0.27148438],
       [-0.01647949],
       [ 0.51953125],
       [ 0.55029297],
       [ 0.00183678],
       [ 0.24829102],
       [ 1.08105469],
       [ 0.26977539],
       [ 0.25878906],
       [ 0.79101562],
       [ 0.83886719],
       [ 0.76025391],
       [ 0.43896484],
       [ 0.31665039],
       [-0.02099609],
       [ 0.62060547],
       [ 0.39941406],
       [ 0.50878906],
       [ 0.26269531],
       [ 0.18322754],
       [ 0.23303223],
       [ 0.57470703],
       [-0.02856445],
       [-0.04156494],
       [ 0.00892639],
       [-0.02537537],
       [ 0.71386719],
       [ 0.30175781],
       [ 0.03567505],
       [ 0.70947266],
       [ 0.50195312],
       [ 0.41113281],
       [ 0.24768066]])

In [26]:
preds = np.clip(preds, 0, 1)
preds

array([[0.5703125 ],
       [0.72070312],
       [0.55126953],
       [0.27148438],
       [0.        ],
       [0.51953125],
       [0.55029297],
       [0.00183678],
       [0.24829102],
       [1.        ],
       [0.26977539],
       [0.25878906],
       [0.79101562],
       [0.83886719],
       [0.76025391],
       [0.43896484],
       [0.31665039],
       [0.        ],
       [0.62060547],
       [0.39941406],
       [0.50878906],
       [0.26269531],
       [0.18322754],
       [0.23303223],
       [0.57470703],
       [0.        ],
       [0.        ],
       [0.00892639],
       [0.        ],
       [0.71386719],
       [0.30175781],
       [0.03567505],
       [0.70947266],
       [0.50195312],
       [0.41113281],
       [0.24768066]])

In [27]:
classes = np.array([0.0, 0.25, 0.50, 0.75, 1.0])
preds = preds.flatten()  
preds = classes[np.abs(classes[:, None] - preds).argmin(axis=0)]

In [28]:
preds

array([0.5 , 0.75, 0.5 , 0.25, 0.  , 0.5 , 0.5 , 0.  , 0.25, 1.  , 0.25,
       0.25, 0.75, 0.75, 0.75, 0.5 , 0.25, 0.  , 0.5 , 0.5 , 0.5 , 0.25,
       0.25, 0.25, 0.5 , 0.  , 0.  , 0.  , 0.  , 0.75, 0.25, 0.  , 0.75,
       0.5 , 0.5 , 0.25])